In [1]:
#importing libraries
import numpy as np
import pandas as pd
from datetime import datetime
import os
#keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split 
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from datetime import datetime
from packaging import version
#################TRAINING PARAM
verbose_level=0

from tensorboard.plugins.hparams import api as hp
import threading

jobs=1

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from keras import regularizers
from keras.layers import Dropout

# for combining all the hyper-parameters
import itertools 
from time import time
from functools import reduce 
import operator

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


#jobs=1 #due to the low dimension of the dataset the single gpu version is slower, so I'm forcing it to the multi core version


In [2]:

g_decay = [0.01] # more estimations can be added





# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([10, 20, 40, 60, 80, 100]))
# HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([4,8,16, 32]))
# HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([4,8,16, 32]))
# HP_ACTIVATION1 = hp.HParam('activation1', hp.Discrete(["relu", "tanh","sigmoid","softmax"]))
# HP_ACTIVATION2 = hp.HParam('activation2', hp.Discrete(["relu", "tanh","sigmoid","softmax"]))
# HP_ACTIVATION3 = hp.HParam('activation3', hp.Discrete(["relu", "tanh","sigmoid","softmax"]))
# HP_INIT = hp.HParam('init_mode', hp.Discrete(['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']))
# HP_MOMENTUM = hp.HParam('momentum', hp.Discrete(np.arange(0.1, 0.9, 0.1).tolist()))
# HP_LEARNING_RATE = hp.HParam('lr', hp.Discrete(np.arange(0.1, 0.9, 0.1).tolist()))
# HP_DROPOUT1 = hp.HParam('dropout1', hp.Discrete(np.arange(0.1, 0.9, 0.1).tolist()))
# HP_DROPOUT2 = hp.HParam('dropout2', hp.Discrete(np.arange(0.1, 0.9, 0.1).tolist()))
# #HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_EPOCH = hp.HParam('epoch',hp.Discrete(range(50,250,50)))


HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([50]))
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([4,8]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([4,8]))
HP_ACTIVATION1 = hp.HParam('activation1', hp.Discrete(["relu", "tanh"]))
HP_ACTIVATION2 = hp.HParam('activation2', hp.Discrete(["relu", "tanh"]))
HP_ACTIVATION3 = hp.HParam('activation3', hp.Discrete(["relu", "tanh"]))
HP_INIT = hp.HParam('init_mode', hp.Discrete(['uniform']))
HP_MOMENTUM = hp.HParam('momentum', hp.Discrete(np.arange(0.1, 0.9, 0.5).tolist()))
HP_LEARNING_RATE = hp.HParam('lr', hp.Discrete(np.arange(0.1, 0.9, 0.5).tolist()))
HP_DROPOUT1 = hp.HParam('dropout1', hp.Discrete(np.arange(0.1, 0.9, 0.5).tolist()))
HP_DROPOUT2 = hp.HParam('dropout2', hp.Discrete(np.arange(0.1, 0.9, 0.5).tolist()))
#HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_EPOCH = hp.HParam('epoch',hp.Discrete(range(150,250,100)))


METRIC_ACCURACY = 'accuracy'


with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[
        HP_BATCH_SIZE,
        HP_NUM_UNITS1,
        HP_NUM_UNITS2,
        HP_ACTIVATION1,
        HP_ACTIVATION2,
        HP_ACTIVATION3,
        HP_INIT,
        HP_MOMENTUM,
        HP_LEARNING_RATE,
        #HP_OPTIMIZER,
        HP_DROPOUT1,
        HP_DROPOUT2,
        HP_EPOCH
    ],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [3]:
%load_ext tensorboard

In [4]:
# reading training and testing data from the csv file
trainDf = pd.read_csv ('data/csv/monks-3Train.csv',header=None).to_numpy()
testDf = pd.read_csv ('data/csv/monks-3Test.csv',header=None).to_numpy()
trainDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)
testDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)

In [5]:
#Preparing trainDf for training
trainX = trainDf[:, 1:7]
trainY = trainDf[:, 0]
#Preparing testDf for validation
testX = testDf[:, 1:7]
testY = testDf[:, 0]
#trainX = scaler.fit_transform(trainX)
#trainY = scaler.fit_transform(trainY)

In [6]:
import keras as ks
tensorboard_callback = ks.callbacks.TensorBoard(log_dir="./log",histogram_freq=1,write_graph=True,update_freq=1)


In [7]:
def create_model(lr=0.1,momentum=0.9,dropout1=0.4,dropout2=0.4,g_hiddeLayerunit1=4,g_hiddeLayerunit2=4,activation1="tanh",activation2="tanh",activation3="sigmoid",init_mode='uniform', g_decay=0.01):
    model = Sequential()
    model.add(Dense(g_hiddeLayerunit1, input_dim=6, kernel_initializer=init_mode, activation=activation1))
    model.add(Dropout(dropout1)),
    model.add(Dense(g_hiddeLayerunit2, kernel_initializer=init_mode, activation=activation2,kernel_regularizer=regularizers.l1(0.01))) # l1 regularization parameter could also be something we can also grid search for.
    model.add(Dropout(dropout2)),
    model.add(Dense(1, activation3))
    sgd = SGD(lr=lr, momentum=momentum, decay=g_decay, nesterov=False,)  # We can add decay to hyper parameter list to get optimum value. 
    model.compile(optimizer=sgd, loss='mean_squared_error',metrics=['accuracy'])
    return model


In [10]:
def train_test_model(hparams,tensorboard_callback):
    
    
  model = create_model(lr=hparams[HP_LEARNING_RATE],momentum=hparams[HP_MOMENTUM],dropout1=hparams[HP_DROPOUT1],dropout2=hparams[HP_DROPOUT2],g_hiddeLayerunit1=hparams[HP_NUM_UNITS1],g_hiddeLayerunit2=hparams[HP_NUM_UNITS1],activation1=hparams[HP_ACTIVATION1],activation2=hparams[HP_ACTIVATION2],activation3=hparams[HP_ACTIVATION3],init_mode=hparams[HP_INIT], g_decay=0.01)


  model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(testX, testY)
  return accuracy

def run(run_dir,hparams,tensorboard_callback):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams,tensorboard_callback)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

session_num = 0
class myThread (threading.Thread):
   def __init__(self, threadID,run_dir,hparams,tensorboard_callback):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.run_dir = run_dir
      self.hparams = hparams
      self.tensorboard_callback=tensorboard_callback
   def run(self):
      print("Starting " + self.name)
      run(self.run_dir,self.hparams,self.tensorboard_callback)
      print("Exiting " + self.name)


for batch_size in HP_BATCH_SIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
      for num_units2 in HP_NUM_UNITS2.domain.values:
        for activation1 in HP_ACTIVATION1.domain.values:
          for activation2 in HP_ACTIVATION2.domain.values:
            for activation3 in HP_ACTIVATION3.domain.values:
              for init_mode in HP_INIT.domain.values:
                for momentum in HP_MOMENTUM.domain.values:
                  for lr in HP_LEARNING_RATE.domain.values:
                      for dropout_rate1 in HP_DROPOUT1.domain.values:
                        for dropout_rate2 in HP_DROPOUT2.domain.values:
                        #for optimizer in HP_OPTIMIZER.domain.values:
                            for epoch in HP_EPOCH.domain.values:
                              hparams = {
                                  HP_NUM_UNITS1: num_units1,
                                  HP_NUM_UNITS2: num_units2,
                                  HP_ACTIVATION1:activation1,
                                  HP_ACTIVATION2:activation2,
                                  HP_ACTIVATION3:activation3,
                                  HP_INIT:init_mode,
                                  HP_MOMENTUM:momentum,
                                  HP_LEARNING_RATE:lr,
                                  HP_DROPOUT1: dropout_rate1,
                                  HP_DROPOUT2: dropout_rate2,
                                  #HP_OPTIMIZER: optimizer,
                                  HP_EPOCH:epoch,
                                  HP_BATCH_SIZE:batch_size,
                              }
                              run_name = "run-%d" % session_num
                              tensorboard_callback = ks.callbacks.TensorBoard(log_dir="logs/scalar/"+run_name,histogram_freq=1,write_graph=True,update_freq=1)
                              print('--- Starting trial: %s' % run_name)
                              print({h.name: hparams[h] for h in hparams})
                              thread=myThread(session_num,'logs/hparam_tuning/' + run_name, hparams,tensorboard_callback)
                              thread.start()
                              session_num += 1





# for batch_size in HP_BATCH_SIZE.domain.values:
#     for num_units1 in HP_NUM_UNITS1.domain.values:
#       for num_units2 in HP_NUM_UNITS2.domain.values:
#         for activation1 in HP_ACTIVATION1.domain.values:
#           for activation2 in HP_ACTIVATION2.domain.values:
#             for activation3 in HP_ACTIVATION3.domain.values:
#               for init_mode in HP_INIT.domain.values:
#                 for momentum in HP_MOMENTUM.domain.values:
#                   for lr in HP_LEARNING_RATE.domain.values:
#                       for dropout_rate1 in HP_DROPOUT1.domain.values:
#                         for dropout_rate2 in HP_DROPOUT2.domain.values:
#                         #for optimizer in HP_OPTIMIZER.domain.values:
#                             for epoch in HP_EPOCH.domain.values:
#                               hparams = {
#                                   HP_NUM_UNITS1: num_units1,
#                                   HP_NUM_UNITS2: num_units2,
#                                   HP_ACTIVATION1:activation1,
#                                   HP_ACTIVATION2:activation2,
#                                   HP_ACTIVATION3:activation3,
#                                   HP_INIT:init_mode,
#                                   HP_MOMENTUM:momentum,
#                                   HP_LEARNING_RATE:lr,
#                                   HP_DROPOUT1: dropout_rate1,
#                                   HP_DROPOUT2: dropout_rate2,
#                                   #HP_OPTIMIZER: optimizer,
#                                   HP_EPOCH:epoch,
#                                   HP_BATCH_SIZE:batch_size,
#                               }
#                               run_name = "run-%d" % session_num
#                               tensorboard_callback = ks.callbacks.TensorBoard(log_dir="logs/scalar/"+run_name,histogram_freq=1,write_graph=True,update_freq=1)
#                               print('--- Starting trial: %s' % run_name)
#                               print({h.name: hparams[h] for h in hparams})
#                               run('logs/hparam_tuning/' + run_name, hparams,tensorboard_callback)
#                               session_num += 1

%tensorboard --logdir logs

--- Starting trial: run-0
{'num_units1': 4, 'num_units2': 4, 'activation1': 'relu', 'activation2': 'relu', 'activation3': 'relu', 'init_mode': 'uniform', 'momentum': 0.1, 'lr': 0.1, 'dropout1': 0.1, 'dropout2': 0.1, 'epoch': 150, 'batch_size': 50}
Starting Thread-521
--- Starting trial: run-1
{'num_units1': 4, 'num_units2': 4, 'activation1': 'relu', 'activation2': 'relu', 'activation3': 'relu', 'init_mode': 'uniform', 'momentum': 0.1, 'lr': 0.1, 'dropout1': 0.1, 'dropout2': 0.6, 'epoch': 150, 'batch_size': 50}
Starting Thread-522
--- Starting trial: run-2
{'num_units1': 4, 'num_units2': 4, 'activation1': 'relu', 'activation2': 'relu', 'activation3': 'relu', 'init_mode': 'uniform', 'momentum': 0.1, 'lr': 0.1, 'dropout1': 0.6, 'dropout2': 0.1, 'epoch': 150, 'batch_size': 50}
Starting Thread-523
--- Starting trial: run-3
{'num_units1': 4, 'num_units2': 4, 'activation1': 'relu', 'activation2': 'relu', 'activation3': 'relu', 'init_mode': 'uniform', 'momentum': 0.1, 'lr': 0.1, 'dropout1': 0.

AlreadyExistsError: Another profiler is running.

Exception in thread Thread-186:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-9-65611f822856>", line 27, in run
    run(self.run_dir,self.hparams,self.tensorboard_callback)
  File "<ipython-input-9-65611f822856>", line 14, in run
    accuracy = train_test_model(hparams,tensorboard_callback)
  File "<ipython-input-9-65611f822856>", line 7, in train_test_model
    model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1097, in fit
    callbacks.on_train_batch_begin(step)
  Fil

Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-9-65611f822856>", line 27, in run
    run(self.run_dir,self.hparams,self.tensorboard_callback)
  File "<ipython-input-9-65611f822856>", line 14, in run
    accuracy = train_test_model(hparams,tensorboard_callback)
  File "<ipython-input-9-65611f822856>", line 7, in train_test_model
    model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1097, in fit
    callbacks.on_train_batch_begin(step)
  File

Exception in thread Thread-429:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-9-65611f822856>", line 27, in run
    run(self.run_dir,self.hparams,self.tensorboard_callback)
  File "<ipython-input-9-65611f822856>", line 14, in run
    accuracy = train_test_model(hparams,tensorboard_callback)
  File "<ipython-input-9-65611f822856>", line 7, in train_test_model
    model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1097, in fit
    callbacks.on_train_batch_begin(step)
  Fil

...

Exception in thread Thread-857:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-10-65611f822856>", line 27, in run
    run(self.run_dir,self.hparams,self.tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 14, in run
    accuracy = train_test_model(hparams,tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 7, in train_test_model
    model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1097, in fit
    callbacks.on_train_batch_begin(step)
  

Exception in thread Thread-895:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-10-65611f822856>", line 27, in run
    run(self.run_dir,self.hparams,self.tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 14, in run
    accuracy = train_test_model(hparams,tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 7, in train_test_model
    model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1097, in fit
    callbacks.on_train_batch_begin(step)
  

Exception in thread Thread-904:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-10-65611f822856>", line 27, in run
    run(self.run_dir,self.hparams,self.tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 14, in run
    accuracy = train_test_model(hparams,tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 7, in train_test_model
    model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1097, in fit
    callbacks.on_train_batch_begin(step)
  

Exception in thread Thread-915:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-10-65611f822856>", line 27, in run
    run(self.run_dir,self.hparams,self.tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 14, in run
    accuracy = train_test_model(hparams,tensorboard_callback)
  File "<ipython-input-10-65611f822856>", line 7, in train_test_model
    model.fit(trainX, trainY, epochs=hparams[HP_EPOCH], callbacks=[tensorboard_callback],verbose=0) # Run with 1 epoch to speed things up for demo purposes
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\branch-env\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1097, in fit
    callbacks.on_train_batch_begin(step)
  

4/4 [==============================] - 0s 3ms/step - loss: 0.2500 - accuracy: 0.5082
Exiting Thread-64
4/4 [==============================] - 0s 2ms/step - loss: 0.1486 - accuracy: 0.8115
Exiting Thread-233


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs


4/4 [==============================] - 0s 1ms/step - loss: 0.2396 - accuracy: 0.7951
Exiting Thread-8
